In [31]:
import pandas as pd
import numpy as np

df = pd.read_excel("GA_task.xlsx")
df.head(5)

,1,Unnamed: 1,2,Unnamed: 3,3,Unnamed: 5,4,Unnamed: 7,5,Unnamed: 9,...,46,Unnamed: 91,47,Unnamed: 93,48,Unnamed: 95,49,Unnamed: 97,50,Unnamed: 99
0,R,T,R,T,R,T,R,T,R,T,...,R,T,R,T,R,T,R,T,R,T
1,9,22,5,17,5,24,8,40,9,14,...,7,21,9,24,6,37,9,26,10,36
2,3,49,10,13,4,28,5,47,7,45,...,3,10,10,40,7,22,1,39,2,46
3,1,47,7,23,1,18,5,47,6,30,...,8,22,9,24,3,39,6,31,4,16
4,9,30,4,29,5,32,7,18,2,20,...,7,27,7,22,2,49,10,13,5,18


In [32]:
tasks = []

for i in range(0,len(df.columns), 2):
    col1 = df.iloc[:, i]
    resource = col1.tolist()[1:]
    if i+1 < len(df.columns):
        col2 = df.iloc[:, i+1]
        time = col2.tolist()[1:]
    tasks.append(list(zip(resource, time)))    


In [33]:
tasks

[[(9, 22),
  (3, 49),
  (1, 47),
  (9, 30),
  (5, 21),
  (3, 19),
  (2, 18),
  (7, 45),
  (6, 10),
  (8, 39),
  (5, 46)],
 [(5, 17),
  (10, 13),
  (7, 23),
  (4, 29),
  (6, 27),
  (8, 33),
  (2, 49),
  (9, 39),
  (1, 15),
  (9, 12),
  (7, 13)],
 [(5, 24),
  (4, 28),
  (1, 18),
  (5, 32),
  (4, 17),
  (10, 31),
  (8, 50),
  (7, 34),
  (5, 38),
  (1, 29),
  (1, 26)],
 [(8, 40),
  (5, 47),
  (5, 47),
  (7, 18),
  (7, 27),
  (2, 43),
  (7, 33),
  (1, 10),
  (1, 35),
  (7, 15),
  (9, 39)],
 [(9, 14),
  (7, 45),
  (6, 30),
  (2, 20),
  (10, 42),
  (1, 29),
  (4, 21),
  (9, 35),
  (7, 43),
  (1, 12),
  (7, 36)],
 [(6, 33),
  (3, 50),
  (9, 19),
  (3, 28),
  (3, 25),
  (5, 26),
  (7, 30),
  (5, 34),
  (5, 15),
  (7, 12),
  (9, 35)],
 [(10, 48),
  (4, 17),
  (7, 40),
  (6, 44),
  (8, 35),
  (6, 16),
  (3, 47),
  (7, 43),
  (8, 47),
  (3, 17),
  (4, 17)],
 [(3, 32),
  (3, 36),
  (1, 36),
  (2, 21),
  (10, 21),
  (8, 48),
  (7, 47),
  (8, 23),
  (4, 40),
  (6, 18),
  (5, 15)],
 [(4, 42),
  (3, 21

In [42]:
import random

POPULATION_SIZE = 100  
NUM_GENERATIONS = 200  
MUTATION_RATE = 0.2    
TOURNAMENT_SIZE = 5
ELITISM = True
ELITISM_COUNT = 2     

# Fitness function
def calculate_fitness(chromosome):
    # Simulate the scheduling to compute total time
    num_resources = max(max(resource for (resource, time) in task) for task in tasks) + 1
    resource_end_time = [0] * num_resources
    
    task_end_time = [0] * len(chromosome)  # Track end times for each task

    # Traverse tasks in the order provided by the chromosome
    for task_index in chromosome:
        start_time = 0  # The time when the current task can start
        for resource, time in tasks[task_index]:
            start_time = max(start_time, resource_end_time[resource])
            resource_end_time[resource] = start_time + time
        
        # Record end time of this task
        task_end_time[task_index] = start_time

    makespan = max(resource_end_time)
    return 1 / makespan, makespan  # Minimize makespan, maximize fitness

# Selection - Tournament selection
def tournament_selection(population, fitness_scores):
    selected = []
    for _ in range(TOURNAMENT_SIZE):
        i = random.randint(0, len(population) - 1)
        selected.append((population[i], fitness_scores[i]))
    return max(selected, key=lambda x: x[1][0])[0]

# Crossover - Ordered Crossover (OX)
def crossover(parent1, parent2):
    size = len(parent1)
    p1 = [-1]*size

    # Random crossover points
    start, end = sorted([random.randint(0, size) for _ in range(2)])
    p1[start:end] = parent1[start:end]

    # Fill the remaining positions with the order in which they appear in the parent
    current_pos = end
    for val in parent2:
        if val not in p1:
            if current_pos >= size:
                current_pos = 0
            p1[current_pos] = val
            current_pos += 1
    return p1

# Mutation - Swap two tasks
def mutate(chromosome):
    if random.random() < MUTATION_RATE:
        i, j = random.sample(range(len(chromosome)), 2)
        chromosome[i], chromosome[j] = chromosome[j], chromosome[i]

# Main Genetic Algorithm
def genetic_algorithm(ELITISM):
    # Initialize population randomly
    population = [random.sample(range(len(tasks)), len(tasks)) for _ in range(POPULATION_SIZE)]

    for generation in range(NUM_GENERATIONS):
        # Calculate fitness scores
        fitness_scores = [calculate_fitness(chromosome) for chromosome in population]

        # New generation with elitism
        if ELITISM:
            new_population = sorted(population, key=lambda ind: calculate_fitness(ind)[0], reverse=True)[:ELITISM_COUNT]
        elif not ELITISM:
            new_population = []

        for _ in range(POPULATION_SIZE - ELITISM_COUNT):
            # Selection
            parent1 = tournament_selection(population, fitness_scores)
            parent2 = tournament_selection(population, fitness_scores)

            # Crossover
            offspring = crossover(parent1, parent2)

            # Mutation
            mutate(offspring)

            new_population.append(offspring)

        population = new_population

        # Track the best solution
        best_individual = max(population, key=lambda ind: calculate_fitness(ind)[0])
        best_fitness, best_makespan = calculate_fitness(best_individual)
        
        if generation % 20 == 0:
            print(f"Generation {generation}: Best Fitness = {best_fitness:.4f}, Best Makespan = {best_makespan:.2f}")

    # Find the best solution in the final population
    best_solution = max(population, key=lambda ind: calculate_fitness(ind)[0])
    best_fitness, best_makespan = calculate_fitness(best_solution)
    print(f"\nBest solution found: {best_solution}")
    print(f"Shortest time to complete all tasks (Makespan): {best_makespan:.2f}")
    


In [43]:
ELITISM = True
genetic_algorithm(ELITISM=True)


Generation 0: Best Fitness = 0.0003, Best Makespan = 3694.00
Generation 20: Best Fitness = 0.0003, Best Makespan = 3217.00
Generation 40: Best Fitness = 0.0003, Best Makespan = 3012.00
Generation 60: Best Fitness = 0.0003, Best Makespan = 2980.00
Generation 80: Best Fitness = 0.0003, Best Makespan = 2927.00
Generation 100: Best Fitness = 0.0003, Best Makespan = 2927.00
Generation 120: Best Fitness = 0.0003, Best Makespan = 2904.00
Generation 140: Best Fitness = 0.0003, Best Makespan = 2894.00
Generation 160: Best Fitness = 0.0004, Best Makespan = 2838.00
Generation 180: Best Fitness = 0.0004, Best Makespan = 2818.00

Best solution found: [34, 24, 17, 39, 0, 14, 40, 13, 1, 38, 18, 15, 21, 6, 23, 28, 48, 25, 49, 35, 32, 16, 41, 27, 29, 3, 8, 43, 46, 45, 22, 26, 12, 4, 7, 30, 31, 11, 2, 37, 9, 10, 42, 33, 5, 36, 20, 19, 47, 44]
Shortest time to complete all tasks (Makespan): 2738.00


In [36]:
ELITISM = False
genetic_algorithm(ELITISM)

Generation 0: Best Fitness = 0.0003, Best Makespan = 3787.00
Generation 1: Best Fitness = 0.0003, Best Makespan = 3668.00
Generation 2: Best Fitness = 0.0003, Best Makespan = 3738.00
Generation 3: Best Fitness = 0.0003, Best Makespan = 3685.00
Generation 4: Best Fitness = 0.0003, Best Makespan = 3670.00
Generation 5: Best Fitness = 0.0003, Best Makespan = 3606.00
Generation 6: Best Fitness = 0.0003, Best Makespan = 3519.00
Generation 7: Best Fitness = 0.0003, Best Makespan = 3504.00
Generation 8: Best Fitness = 0.0003, Best Makespan = 3414.00
Generation 9: Best Fitness = 0.0003, Best Makespan = 3406.00
Generation 10: Best Fitness = 0.0003, Best Makespan = 3468.00
Generation 11: Best Fitness = 0.0003, Best Makespan = 3385.00
Generation 12: Best Fitness = 0.0003, Best Makespan = 3366.00
Generation 13: Best Fitness = 0.0003, Best Makespan = 3331.00
Generation 14: Best Fitness = 0.0003, Best Makespan = 3305.00
Generation 15: Best Fitness = 0.0003, Best Makespan = 3257.00
Generation 16: Bes

([11,
  2,
  21,
  29,
  15,
  6,
  34,
  23,
  13,
  22,
  49,
  38,
  47,
  19,
  12,
  44,
  3,
  32,
  41,
  25,
  45,
  28,
  43,
  4,
  8,
  9,
  33,
  36,
  20,
  1,
  26,
  5,
  30,
  18,
  14,
  42,
  24,
  17,
  10,
  48,
  37,
  27,
  16,
  39,
  35,
  7,
  46,
  40,
  0,
  31],
 2968)

In [37]:
import random

# Sample data
tasks = tasks

# Genetic Algorithm Parameters
POPULATION_SIZE = 100  
NUM_GENERATIONS = 200  
MUTATION_RATE = 0.2    
TOURNAMENT_SIZE = 5

# Fitness function
def calculate_fitness(chromosome):
    # Simulate the scheduling to compute total time
    num_resources = max(max(resource for (resource, time) in task) for task in tasks) + 1
    resource_end_time = [0] * num_resources
    
    task_end_time = [0] * len(chromosome)  # Track end times for each task

    # Traverse tasks in the order provided by the chromosome
    for task_index in chromosome:
        start_time = 0  # The time when the current task can start
        for resource, time in tasks[task_index]:
            start_time = max(start_time, resource_end_time[resource])
            resource_end_time[resource] = start_time + time
        
        # Record end time of this task
        task_end_time[task_index] = start_time

    makespan = max(resource_end_time)
    return 1 / makespan, makespan  # Minimize makespan, maximize fitness

# Selection - Tournament selection
def tournament_selection(population, fitness_scores):
    selected = []
    for _ in range(TOURNAMENT_SIZE):
        i = random.randint(0, len(population) - 1)
        selected.append((population[i], fitness_scores[i]))
    return max(selected, key=lambda x: x[1][0])[0]

# Crossover - Ordered Crossover (OX)
def crossover(parent1, parent2):
    size = len(parent1)
    p1 = [-1]*size

    # Random crossover points
    start, end = sorted([random.randint(0, size) for _ in range(2)])
    p1[start:end] = parent1[start:end]

    # Fill the remaining positions with the order in which they appear in the parent
    current_pos = end
    for val in parent2:
        if val not in p1:
            if current_pos >= size:
                current_pos = 0
            p1[current_pos] = val
            current_pos += 1
    return p1

# Mutation - Swap two tasks
def mutate(chromosome):
    if random.random() < MUTATION_RATE:
        i, j = random.sample(range(len(chromosome)), 2)
        chromosome[i], chromosome[j] = chromosome[j], chromosome[i]

# Main Genetic Algorithm
def genetic_algorithm(elitism):
    # Initialize population randomly
    population = [random.sample(range(len(tasks)), len(tasks)) for _ in range(POPULATION_SIZE)]

    for generation in range(NUM_GENERATIONS):
        # Calculate fitness scores
        fitness_scores = [calculate_fitness(chromosome) for chromosome in population]

        if elitism:
            
        new_population = []  # Initialize new population without elitism

        for _ in range(POPULATION_SIZE):
            # Selection
            parent1 = tournament_selection(population, fitness_scores)
            parent2 = tournament_selection(population, fitness_scores)

            # Crossover
            offspring = crossover(parent1, parent2)

            # Mutation
            mutate(offspring)

            new_population.append(offspring)

        population = new_population

        # Track the best solution
        best_individual = max(population, key=lambda ind: calculate_fitness(ind)[0])
        best_fitness, best_makespan = calculate_fitness(best_individual)
        print(f"Generation {generation}: Best Fitness = {best_fitness:.4f}, Best Makespan = {best_makespan:.2f}")

    # Find the best solution in the final population
    best_solution = max(population, key=lambda ind: calculate_fitness(ind)[0])
    best_fitness, best_makespan = calculate_fitness(best_solution)
    print(f"Best solution found: {best_solution}")
    print(f"Shortest time to complete all tasks (Makespan): {best_makespan:.2f}")
    return best_solution, best_makespan

# Run the genetic algorithm
genetic_algorithm()


IndentationError: expected an indented block after 'if' statement on line 75 (3178688967.py, line 77)